In [43]:
import numpy as np
import pandas as pd
df = pd.read_pickle('data/iemo.pkl')
df

,talker,sentiment_label,dialog,text_w_one_prev,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth,emotion,emotion_label,index,split,text,concatenated
0,MARY,frustrated,7,MARY: What's he going to say?,0,"(6.21, 9.32)","[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,What's he going to say?,16,MARY: What's he going to say?,0,frustrated,Ses01F_script01_1,train,</s></s> MARY: What's he going to say?,</s></s> What's he going to say?
1,JAMES,neutral,7,MARY: What's he going to say? </s></s> JAMES: ...,1,"(11.9892, 15.1219)","[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He saw it.,16,JAMES: He saw it.,1,neutral,Ses01F_script01_1,train,MARY: What's he going to say? </s></s> JAMES: ...,What's he going to say? </s></s> He saw it.
2,JAMES,neutral,7,JAMES: He saw it. </s></s> JAMES: He was out h...,1,"(19.0456, 22.3962)","[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He was out here when it broke.,16,JAMES: He was out here when it broke.,1,neutral,Ses01F_script01_1,train,MARY: What's he going to say? JAMES: He saw it...,What's he going to say? He saw it. </s></s> ...
3,JAMES,neutral,7,JAMES: He was out here when it broke. </s></s>...,1,"(24.8, 35.9971)","[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,Four o clock this morning. I heard a crack a...,16,JAMES: Four o clock this morning. I heard a c...,1,neutral,Ses01F_script01_1,train,MARY: What's he going to say? JAMES: He saw it...,What's he going to say? He saw it. He was o...
4,JAMES,neutral,7,JAMES: Four o clock this morning. I heard a c...,1,"(38.66, 40.53)","[[[27, 35, 168, 218], [184, 129, 222, 205], [0...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,I don't know.,16,JAMES: I don't know.,1,neutral,Ses01F_script01_1,train,MARY: What's he going to say? JAMES: He saw it...,What's he going to say? He saw it. He was o...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,WILLIAM,excited,149,"ELIZABETH: Oh, It burnt my comb and all the to...",4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,excited,Ses05M_script03_1,test,"WILLIAM: I wonder. ELIZABETH: what, Nobody kno...","I wonder. what, Nobody knows we are here exc..."
7376,WILLIAM,excited,149,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard.",4,excited,Ses05M_script03_1,test,"WILLIAM: I wonder. ELIZABETH: what, Nobody kno...","I wonder. what, Nobody knows we are here exc..."
7377,WILLIAM,excited,149,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...,4,excited,Ses05M_script03_1,test,"WILLIAM: I wonder. ELIZABETH: what, Nobody kno...","I wonder. what, Nobody knows we are here exc..."
7378,WILLIAM,excited,149,WILLIAM: I should never forget his face. [LAUG...,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We

# Splits

In [27]:
df['split'].value_counts()

split
train    4433
test     1475
val      1472
Name: count, dtype: int64

OLD SPLITS:

split
train    4433
test     1475
val      1472
Name: count, dtype: int64

So use different splits like with EmoBERTa. Our text is still fine, just the keys of hdf5 will be messed up, but might be easy to change in code


In [29]:
splits = pd.read_json('data/utterance-ordered.json')

splits['test'] = splits['test'].replace({np.nan:None})
splits['train'] = splits['train'].replace({np.nan:None})
splits['val'] = splits['val'].replace({np.nan:None})
splits['split'] = splits.apply(lambda x: "test" if x['test'] != None else "train" if x['train'] != None else "val" , axis=1)
splits.drop(columns=['test','train','val'], inplace=True)
splits.reset_index(inplace=True)

df['index'] = df['audio_path'].apply(lambda x: x.split('/')[-1].split('.')[0])
df.drop(columns=['split'], inplace=True)
df['split'] = df['index'].apply(lambda x: splits[splits['index'] == x]['split'].values[0])
# df.drop(columns=['index'], inplace=True)
df

,talker,sentiment_label,dialog,text,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth,emotion,emotion_label,index,split
0,MARY,frustrated,7,MARY: What's he going to say?,0,"(6.21, 9.32)","[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,What's he going to say?,16,MARY: What's he going to say?,0,frustrated,Ses01F_script01_1,train
1,JAMES,neutral,7,MARY: What's he going to say? </s></s> JAMES: ...,1,"(11.9892, 15.1219)","[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He saw it.,16,JAMES: He saw it.,1,neutral,Ses01F_script01_1,train
2,JAMES,neutral,7,JAMES: He saw it. </s></s> JAMES: He was out h...,1,"(19.0456, 22.3962)","[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He was out here when it broke.,16,JAMES: He was out here when it broke.,1,neutral,Ses01F_script01_1,train
3,JAMES,neutral,7,JAMES: He was out here when it broke. </s></s>...,1,"(24.8, 35.9971)","[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,Four o clock this morning. I heard a crack a...,16,JAMES: Four o clock this morning. I heard a c...,1,neutral,Ses01F_script01_1,train
4,JAMES,neutral,7,JAMES: Four o clock this morning. I heard a c...,1,"(38.66, 40.53)","[[[27, 35, 168, 218], [184, 129, 222, 205], [0...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,I don't know.,16,JAMES: I don't know.,1,neutral,Ses01F_script01_1,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,WILLIAM,excited,149,"ELIZABETH: Oh, It burnt my comb and all the to...",4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,excited,Ses05M_script03_1,test
7376,WILLIAM,excited,149,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard.",4,excited,Ses05M_script03_1,test
7377,WILLIAM,excited,149,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...,4,excited,Ses05M_script03_1,test
7378,WILLIAM,excited,149,WILLIAM: I should never forget his face. [LAUG...,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We were very much younger then.,4,excited,Ses05M_script03_1,test


In [30]:
df['split'].value_counts()

split
train    4778
test     1622
val       980
Name: count, dtype: int64

In [31]:
df.to_pickle("data/iemo.pkl")

In [15]:
sessions = df[['index' , 'split']]
sessions

,index,split
0,Ses01F_script01_1,train
1,Ses01F_script01_1,train
2,Ses01F_script01_1,train
3,Ses01F_script01_1,train
4,Ses01F_script01_1,train
...,...,...
7375,Ses05M_script03_1,val
7376,Ses05M_script03_1,val
7377,Ses05M_script03_1,val
7378,Ses05M_script03_1,val


In [16]:
# Group by 'index' and check if all 'split' values are the same for each group
sessions['is_unique'] = sessions.groupby('index')['split'].transform(lambda x: x.nunique() == 1)

# Filter out rows where 'split' values are not unique
sessions = sessions[sessions['is_unique']]
sessions

/tmp/ipykernel_1825724/390534332.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sessions['is_unique'] = sessions.groupby('index')['split'].transform(lambda x: x.nunique() == 1)


,index,split,is_unique
0,Ses01F_script01_1,train,True
1,Ses01F_script01_1,train,True
2,Ses01F_script01_1,train,True
3,Ses01F_script01_1,train,True
4,Ses01F_script01_1,train,True
...,...,...,...
7375,Ses05M_script03_1,val,True
7376,Ses05M_script03_1,val,True
7377,Ses05M_script03_1,val,True
7378,Ses05M_script03_1,val,True


In [18]:
b = True
for bb in sessions['is_unique'].to_list():
    b = b & bb
    
print(b)

True


In [13]:
sessions['index'].drop_duplicates()

0       Ses01F_script01_1
74         Ses01F_impro04
135     Ses01F_script01_3
189     Ses01F_script02_2
261        Ses01F_impro02
              ...        
7102       Ses05F_impro02
7151       Ses05F_impro04
7235    Ses05M_script01_3
7278    Ses05F_script02_2
7337    Ses05M_script03_1
Name: index, Length: 151, dtype: object

In [11]:
sessions['index'].value_counts()

index
Ses04F_impro07       110
Ses04M_script03_2     94
Ses05F_impro05        91
Ses03F_script01_1     87
Ses05F_impro04        84
                    ... 
Ses05M_script01_2     23
Ses04M_script01_3     20
Ses03F_impro01        18
Ses03F_script03_1     18
Ses03M_script03_1      8
Name: count, Length: 151, dtype: int64

# All previous utterances

In [71]:
def concatenate_utterances_with_auth_old(df):
    df['concatenated_with_auth'] = ''
    groups = df.groupby(['split', 'index'])
    for name, group in groups:
        utterances = group['og_text_with_auth'].tolist()
        for i in range(len(utterances)):
            concatenated_with_auth = ' '.join(utterances[:i] + ['</s></s> ' + utterances[i]])
            df.loc[group.index[i], 'concatenated_with_auth'] = concatenated_with_auth
    return df

def concatenate_utterances_with_auth(df):
    df['concatenated_with_auth'] = ''
    groups = df.groupby(['split', 'index'])
    for name, group in groups:
        utterances = group['og_text_with_auth'].tolist()
        for i in range(len(utterances)):
            # Start from the end and keep adding utterances until the total number of words exceeds 512
            concatenated_with_auth = ''
            if i == 0:
                concatenated_with_auth = ' '.join(utterances[:i] + ['</s></s> ' + utterances[i]])
                df.loc[group.index[i], 'concatenated_with_auth'] = concatenated_with_auth
            else:
                for j in range(i, -1, -1):
                    if j == i:
                        temp_concatenated_with_auth = ''.join([ '</s></s> ' + utterances[j]])
                    else:
                        temp_concatenated_with_auth = ' '.join([utterances[j]] + [concatenated_with_auth])
                    if len(temp_concatenated_with_auth.split()) > 500:
                        break
                    concatenated_with_auth = temp_concatenated_with_auth
                df.loc[group.index[i], 'concatenated_with_auth'] = concatenated_with_auth
    return df


def concatenate_utterances(df):
    df['concatenated'] = ''
    groups = df.groupby(['split', 'index'])
    for name, group in groups:
        utterances = group['og_text'].tolist()
        for i in range(len(utterances)):
            concatenated = ' '.join(utterances[:i] + ['</s></s> ' + utterances[i]])
            df.loc[group.index[i], 'concatenated'] = concatenated
    return df
df = concatenate_utterances_with_auth(df)
# df = concatenate_utterances(df)
df['concatenated_with_auth']

0                  </s></s> MARY: What's he going to say?
1       MARY: What's he going to say? </s></s> JAMES: ...
2       MARY: What's he going to say? JAMES: He saw it...
3       MARY: What's he going to say? JAMES: He saw it...
4       MARY: What's he going to say? JAMES: He saw it...
                              ...                        
7375    ELIZABETH: what, Nobody knows we are here exce...
7376    ELIZABETH: What do we do? WILLIAM: We're all r...
7377    WILLIAM: We're all right, aren't we, darling, ...
7378    ELIZABETH: Now and always, my sweet. WILLIAM: ...
7379    WILLIAM: I don't care then. ELIZABETH: It was ...
Name: concatenated_with_auth, Length: 7380, dtype: object

In [66]:
for i in range(74):
    print(df.iloc[i]['concatenated_with_auth'])

</s></s> MARY: What's he going to say?
MARY: What's he going to say? </s></s> JAMES: He saw it.
MARY: What's he going to say? JAMES: He saw it. </s></s> JAMES: He was out here when it broke.
MARY: What's he going to say? JAMES: He saw it. JAMES: He was out here when it broke. </s></s> JAMES: Four o clock this morning.  I heard a crack and I woke up and looked out.  He was standing right over there when it cracked.
MARY: What's he going to say? JAMES: He saw it. JAMES: He was out here when it broke. JAMES: Four o clock this morning.  I heard a crack and I woke up and looked out.  He was standing right over there when it cracked. </s></s> JAMES: I don't know.
MARY: What's he going to say? JAMES: He saw it. JAMES: He was out here when it broke. JAMES: Four o clock this morning.  I heard a crack and I woke up and looked out.  He was standing right over there when it cracked. JAMES: I don't know. </s></s> MARY: He's dreaming of him again, walking around.
MARY: What's he going to say? JAMES:

In [72]:
df.drop(columns="text" , inplace = True)
df.rename(columns={"concatenated_with_auth" : "text"} , inplace = True)
df

,talker,sentiment_label,dialog,text_w_one_prev,sentiment,timings,bbox,audio_path,video_path,og_text,len,og_text_with_auth,emotion,emotion_label,index,split,concatenated,text
0,MARY,frustrated,7,MARY: What's he going to say?,0,"(6.21, 9.32)","[[[65, 33, 200, 217]], [[63, 32, 203, 217]], [...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,What's he going to say?,16,MARY: What's he going to say?,0,frustrated,Ses01F_script01_1,train,</s></s> What's he going to say?,</s></s> MARY: What's he going to say?
1,JAMES,neutral,7,MARY: What's he going to say? </s></s> JAMES: ...,1,"(11.9892, 15.1219)","[[[24, 37, 172, 217], [0, 86, 36, 219], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He saw it.,16,JAMES: He saw it.,1,neutral,Ses01F_script01_1,train,What's he going to say? </s></s> He saw it.,MARY: What's he going to say? </s></s> JAMES: ...
2,JAMES,neutral,7,JAMES: He saw it. </s></s> JAMES: He was out h...,1,"(19.0456, 22.3962)","[[[19, 36, 175, 216], [2, 62, 54, 224], [182, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,He was out here when it broke.,16,JAMES: He was out here when it broke.,1,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. </s></s> ...,MARY: What's he going to say? JAMES: He saw it...
3,JAMES,neutral,7,JAMES: He was out here when it broke. </s></s>...,1,"(24.8, 35.9971)","[[[40, 37, 176, 217], [2, 74, 68, 220], [183, ...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,Four o clock this morning. I heard a crack a...,16,JAMES: Four o clock this morning. I heard a c...,1,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. He was o...,MARY: What's he going to say? JAMES: He saw it...
4,JAMES,neutral,7,JAMES: Four o clock this morning. I heard a c...,1,"(38.66, 40.53)","[[[27, 35, 168, 218], [184, 129, 222, 205], [0...",../../data/iemocap_wav/Ses01F_script01_1.wav,../../data/iemocap_avi/Ses01F_script01_1.avi,I don't know.,16,JAMES: I don't know.,1,neutral,Ses01F_script01_1,train,What's he going to say? He saw it. He was o...,MARY: What's he going to say? JAMES: He saw it...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,WILLIAM,excited,149,"ELIZABETH: Oh, It burnt my comb and all the to...",4,"(215.4, 220.75)","[[[61, 22, 188, 216]], [[60, 20, 187, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"[LAUGHTER] That was quit a rouser, wasn't it?",16,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...","ELIZABETH: what, Nobody knows we are here exce..."
7376,WILLIAM,excited,149,"WILLIAM: [LAUGHTER] That was quit a rouser, wa...",4,"(221.13, 223.83)","[[[61, 16, 189, 216]], [[61, 15, 188, 216]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,"Oh, I didn't hit you very hard.",16,"WILLIAM: Oh, I didn't hit you very hard.",4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...",ELIZABETH: What do we do? WILLIAM: We're all r...
7377,WILLIAM,excited,149,"WILLIAM: Oh, I didn't hit you very hard. </s><...",4,"(229.88, 233.06)","[[[57, 13, 190, 213]], [[56, 16, 190, 215]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,I should never forget his face. [LAUGHTER],16,WILLIAM: I should never forget his face. [LAUG...,4,excited,Ses05M_script03_1,test,"I wonder. what, Nobody knows we are here exc...","WILLIAM: We're all right, aren't we, darling, ..."
7378,WILLIAM,excited,149,WILLIAM: I should never forget his face. [LAUG...,4,"(234.14, 236.66)","[[[62, 11, 190, 215]], [[60, 13, 189, 214]], [...",../../data/iemocap_wav/Ses05M_script03_1.wav,../../data/iemocap_avi/Ses05M_script03_1.avi,We were very much younger then.,16,WILLIAM: We w

In [74]:
df.to_pickle("data/iemo.pkl")

In [73]:
len(df.iloc[60]['text'].split())

495